In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.datasets import cifar10


print(f"Tensorflow {tf.__version__}")

2025-10-24 15:17:01.564183: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 15:17:01.565803: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 15:17:01.776034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 15:17:03.590791: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


https://www.kaggle.com/code/kedarsai/cifar-10-88-accuracy-using-keras#Model-5:-Adding-Batch-Normalization

In [ ]:

def resnet_v1_eembc():

    inputs = Input(shape = (32, 32, 3))
    x = Conv2D(16, kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64,kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    x = Conv2D(64,kernel_size=(5,5),activation='relu',padding = 'same',use_bias=False)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(10,activation='softmax',use_bias=False)(x)


    # Create functional model
    model= Model(inputs, x)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

    # Loss function
    model.compile(
        loss='categorical_crossentropy', # loss function
        optimizer=optimizer, # learning rule
        metrics=['accuracy'] # show accuracy
    )
    print(model.summary())

    return model

In [8]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

In [9]:
model = resnet_v1_eembc()

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=128,                        # batch size
    epochs=20,                              # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save("CIFAR_v2.keras")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 16)     │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 64)     │        25,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 64)       │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        10,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,440 (544.69 KB)

 Trainable params: 139,440 (544.69 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 47s 131ms/step - accuracy: 0.1619 - loss: 2.2144 - val_accuracy: 0.2564 - val_loss: 2.1021
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 62s 177ms/step - accuracy: 0.2383 - loss: 2.0550 - val_accuracy: 0.3014 - val_loss: 2.0041
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 65s 185ms/step - accuracy: 0.2826 - loss: 1.9682 - val_accuracy: 0.3500 - val_loss: 1.9002
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 57s 161ms/step - accuracy: 0.3266 - loss: 1.8644 - val_accuracy: 0.3668 - val_loss: 1.7974
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 33s 93ms/step - accuracy: 0.3564 - loss: 1.7801 - val_accuracy: 0.3964 - val_loss: 1.7164
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 36s 103ms/step - accuracy: 0.3780 - loss: 1.7212 - val_accuracy: 0.4128 - val_loss: 1.6655
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 32s 91ms/step - accuracy: 0.3943 - loss: 1.6696 - val_accuracy: 0.4404 - val_loss: 1.6020
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 34s 97ms/step - accuracy: 0.4078 - loss:

In [6]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [10]:
net = snn_converter.load_keras_model("CIFAR_v2.keras", show_info=True)

* Input layer: input_layer_1, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d_5, (32, 32, 16) 
* MaxPooling2D layer: max_pooling2d_5, (16, 16, 16) 
* Dropout skipped.
* Conv2D layer: conv2d_6, (16, 16, 64) 
* MaxPooling2D layer: max_pooling2d_6, (8, 8, 64) 
* Dropout skipped.
* Conv2D layer: conv2d_7, (8, 8, 64) 
* MaxPooling2D layer: max_pooling2d_7, (4, 4, 64) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: dense_1, 10 
    weights: (10, 1024)
    mean -0.0001721179869491607, std 0.04966263473033905
    min -0.19562436640262604, max 0.1747375726699829



In [16]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [17:32<00:00,  3.51s/it]


In [17]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.55      0.50      0.52        36
           1       0.50      0.75      0.60        24
           2       0.53      0.37      0.43        27
           3       0.33      0.31      0.32        29
           4       0.33      0.26      0.29        23
           5       0.37      0.25      0.30        28
           6       0.56      0.65      0.60        34
           7       0.38      0.37      0.38        27
           8       0.57      0.84      0.68        37
           9       0.69      0.57      0.62        35

    accuracy                           0.50       300
   macro avg       0.48      0.49      0.48       300
weighted avg       0.49      0.50      0.49       300

Test accuracy of the SNN: 0.5033333333333333


In [14]:
model = tf.keras.models.load_model('CIFAR_v2.keras')
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5310 - loss: 1.3338


In [19]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.5310
Test accuracy of the SNN: 0.5033333333333333
